# Notebook para la registración de la distribución de dosis 2D medida (film) con la distribución de dosis de referencia (planificada)

<b><font color='blue'>Importación de bibliotecas</font></b>

In [1]:
import wx

import SimpleITK as sitk

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
from ipywidgets import *

from IPython.display import display
plt.style.use('ggplot')

<b><font color='blue'>Funciones auxiliares</font></b>

In [ ]:
def get_path(wildcard):
    app = wx.App(None)
    style = wx.FD_OPEN | wx.FD_FILE_MUST_EXIST
    dialog = wx.FileDialog(None, 'Open', wildcard=wildcard, style=style)
    if dialog.ShowModal() == wx.ID_OK:
        path = dialog.GetPath()
    else:
        path = None
    dialog.Destroy()
    return path

def get_array_extent(array_size, pixel_spacing, origin):
    # return extent : scalars (left, right, bottom, top)
    x_min = pixel_spacing[0]/2.0 + origin[0] 
    x_max = array_size[1]*pixel_spacing[0] - pixel_spacing[0]/2.0 + origin[0]
    y_min = pixel_spacing[1]/2.0 + origin[1] 
    y_max = array_size[0]*pixel_spacing[1] - pixel_spacing[1]/2.0 + origin[1]
                                                                                                         
    return np.array([x_min, x_max, y_max, y_min])

<b><font color='blue'>Funciones de visualización</font></b>

In [ ]:
def display_array(my_array, array_extent, l_limit = 0.0, u_limit = 300.0):     
    
    plt.figure(figsize = (8, 8))

    plt.imshow(my_array, cmap = 'hot', vmin=l_limit, vmax = u_limit, 
               interpolation = 'none', extent = array_extent)#

    plt.show()
    

def display_arrays_with_alpha_and_shift(plan_array, film_array, 
                              plan_extent, film_extent, 
                              alpha = 0.5,
                              plan_l_limit = 0.0, film_l_limit = 5000.0,           
                              plan_u_limit = 300.0, film_u_limit = 25000.0, 
                              film_shift_x = 0.0, film_shift_y = 0.0):
    global film_shift      
        
    film_shift = np.array([film_shift_x, film_shift_y])
    
    film_extent = [film_extent[0]+film_shift[0], film_extent[1]+film_shift[0], 
                   film_extent[2]+film_shift[1], film_extent[3]+film_shift[1]]

    plt.figure(figsize = (6, 6))

    plt.imshow(film_array,  alpha = 1.0,cmap = 'hot', vmin=film_l_limit, vmax = film_u_limit, 
               interpolation = 'none', extent = film_extent)

    plt.imshow(plan_array, alpha = alpha, cmap = 'gray', vmin=plan_l_limit, vmax = plan_u_limit, 
               interpolation = 'none', extent = plan_extent)
   
    plt.show()

    
def display_arrays_with_alpha(plan_array, film_array, 
                              plan_extent, film_extent, 
                              alpha = 0.5, 
                              plan_c_limit = 300.0, film_c_limit = 300.0):
    global film_shift      
        
    plt.figure(figsize = (6, 6))

    plt.imshow(film_array,  alpha = 1.0,cmap = 'hot', vmin=0.0, vmax = film_c_limit, 
               interpolation = 'none', extent = film_extent)

    plt.imshow(plan_array,alpha = alpha, cmap = 'gray', vmin=0.0, vmax = plan_c_limit, 
               interpolation = 'none', extent = plan_extent)
   
    plt.show()    


## <b><font color='blue'>Procesamiento de la imagen del film</font></b>
<font color='blue'>Utiliza la funcion get_path (que a su vez utiliza de wxpython) para obtener el path del archivo tiff correspondiente al film. Como wx devuelve el path en unicode, se codifica con 'utf-8' para que sitk pueda leer la imagen. Para leer la imagen debe usarse el tipo de variable -1 (unknown) o sitk.sitkVectorInt16 (ya que los films vienen en este tipo de variable).</font>  


<font color='blue'>Para calibrar la imagen (dosimétrica y geométricamente), es conveniente trabajar con arrays de numpy. Para hacer eso, se extrae de la imagen el array con sitk.GetArrayFromImage(). Se extrae directamente el canal rojo (tener en cuenta que el rojo es el de íncide cero porque en numpy se ordenan en RGB).</font>  

In [ ]:
filename_film = get_path('*.tif')
filename_film = filename_film.encode('utf-8')

film_image_color = sitk.ReadImage(filename_film, -1)

film_array = sitk.GetArrayFromImage(film_image_color)[:,:,0]

film_size = film_array.shape


film_pixel_spacing = np.float64(film_image_color.GetSpacing())

# Coordenadas x, y del pixel superior izquierdo, de tal manera que el (0,0) quede en el centro de la imagen
film_origin = np.array([(1 - film_size[1])*film_pixel_spacing[0]/2.0, (1 - film_size[0])*film_pixel_spacing[1]/2.0])

film_extent = get_array_extent(film_size, film_pixel_spacing, film_origin)

## <b><font color='blue'>Procesamiento de la imagen planificada</font></b> 

In [ ]:
# Load dicom 2D distribution (reference image)
filename_plan = get_path('*.dcm')
filename_plan = filename_plan.encode('utf-8')

plan_image_original = sitk.ReadImage(filename_plan, -1) # sitk.sitkVectorInt16

dose_scaling =  np.float32(plan_image_original.GetMetaData('3004|000e')) #dose gird scaling

plan_array = np.float32(sitk.GetArrayFromImage(plan_image_original)[0,:,:])*dose_scaling*100.0 #por 100.0 para pasar a cGy

plan_size = plan_array.shape

plan_pixel_spacing = plan_image_original.GetSpacing()[0:2]

plan_origin = [(1 - plan_size[1])*plan_pixel_spacing[0]/2.0, (1 - plan_size[0])*plan_pixel_spacing[1]/2.0]

plan_extent = get_array_extent(plan_size, plan_pixel_spacing, plan_origin)

# <font color='blue'><b>Registración</font></b>
<font color='blue'>Aproximación manual inicial</font>

In [ ]:
%matplotlib inline

[min_val, max_val] = [film_array.min(), film_array.max()]


interact(display_arrays_with_alpha_and_shift, plan_array = fixed(plan_array), film_array =fixed(film_array), 
                              plan_extent = fixed(plan_extent), film_extent = fixed(film_extent), 
                              alpha = (0.0,1.0,0.01), 
                              plan_l_limit = (0.0,300.0,1.0), film_l_limit = (min_val,max_val,1.0),   
                              plan_u_limit = (0.0,300.0,1.0), film_u_limit = (min_val,max_val,1.0), 
                              film_shift_x =(-200.0,200.0,0.5),film_shift_y =(-200.0,200.0,0.5))

# Siriraj: shift_x = -7.0, shift_y = -18.0

<b><font color='blue'>Recomposición del film y de la planificación como imagenes de ITK</font></b>

<font color='blue'>Construyo una imagen nuevamente con el film calibrado y le paso la metadata (incluyendo el origen calculado)</font>

In [ ]:

film_image = sitk.GetImageFromArray(film_array)

film_image.SetSpacing(film_pixel_spacing)

#vuelvo a calcular film_origin por las dudas se ejecute esta celda mas de una vez (evitando la acumulacion en la suma)
film_origin = np.array([(1 - film_size[1])*film_pixel_spacing[0]/2.0, (1 - film_size[0])*film_pixel_spacing[1]/2.0])

film_origin = film_origin + film_shift

film_image.SetOrigin(film_origin)


# para la imagen planificada

plan_image = sitk.GetImageFromArray(plan_array)

plan_image.SetSpacing(plan_pixel_spacing)

plan_image.SetOrigin(plan_origin)


print film_shift
print film_origin

print film_image.GetPixelIDTypeAsString()
print film_image.GetPixelID()
print plan_image.GetPixelIDTypeAsString()
print plan_image.GetPixelID()

<b><font color='blue'>Interpolación</font></b>

<font color='blue'>(no es necesaria pero ayuda a la calibración)</font>

In [ ]:
#reference_image = image
interpolator = sitk.sitkLinear
identity = sitk.Transform(2, sitk.sitkIdentity)
default_value = 9000.0
reference_image = film_image
image = plan_image
plan_image_resampled = sitk.Resample(image, reference_image, identity,
                          interpolator, default_value)

In [ ]:

plan_array_resampled = sitk.GetArrayFromImage(plan_image_resampled)

plan_size_resampled = plan_array_resampled.shape

plan_pixel_spacing_resampled = plan_image_resampled.GetSpacing()[0:2]

plan_origin_resampled = plan_image_resampled.GetOrigin()

plan_extent_resampled = get_array_extent(plan_size_resampled, plan_pixel_spacing_resampled, plan_origin_resampled)

print plan_image_resampled.GetSize()
print plan_image_resampled.GetSpacing()
print plan_origin_resampled

Las grafico juntas nuevamente solo para chequear la registracion

In [ ]:
%matplotlib inline

plan_extent = get_array_extent(plan_size, plan_pixel_spacing, plan_origin)
film_extent = get_array_extent(film_size, film_pixel_spacing, film_origin)


interact(display_arrays_with_alpha, 
         plan_array = fixed(plan_array_resampled), film_array =fixed(film_array), 
         plan_extent = fixed(plan_extent_resampled), film_extent = fixed(film_extent), 
         alpha = (0.0,1.0,0.01), 
         plan_c_limit = (0.0,300.0,1.0), film_c_limit = (0.0,max_val,1.0)); 
         
        

In [ ]:
norm_ind = film_image.TransformPhysicalPointToIndex([15.0,-25.0])
norm_val = film_image[norm_ind]
print norm_val